In [1]:
open("log/live_log.log","w")

mode = "Live" # Backtesting/Live/Algo

%run "KiteConnect_Library.ipynb"   
warnings.filterwarnings('ignore')



In [2]:
%run "KiteConnect_Strategies_v2.ipynb" 

In [3]:
logger.setLevel(logging.DEBUG)

gc.collect()

7

%%javascript
IPython.notebook.kernel.execute("URL = '" + window.location + "'");

### Filter Data

In [77]:
oi_url = "https://www.nseindia.com/products/content/equities/equities/oi_spurts.htm"

In [87]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
import re
import html5lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def download_from_nse(symbol):
    driver = webdriver.Chrome("./automation/chromedriver")
    driver.get("https://www.nseindia.com/products/content/equities/equities/eq_security.htm")
    #assert "Python" in driver.title
    elem = driver.find_element_by_name("symbol")
    elem.clear()
    elem.send_keys(symbol)

    dateRange = Select(driver.find_element_by_name("dateRange"))
    dateRange.select_by_value("week")


    dateRange = Select(driver.find_element_by_name("series"))
    dateRange.select_by_value("EQ")

    #submitBtn = driver.find_element_by_id("submitMe")
    #elem.submit()

    driver.execute_script("submitData()")

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "download-data-link"))
        )
        oi_html= driver.find_element_by_id('historicalData').get_attribute('innerHTML')

        #f_states=   pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states') 
        stock_data = pd.read_html(oi_html, header=[0])[0]

    finally:
        driver.quit()
        
    return stock_data

In [89]:
import pandas as pd
import html5lib
driver = webdriver.Chrome("./automation/chromedriver")
driver.get(oi_url)

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "tab7Content"))
    )
    oi_html= driver.find_element_by_id('tab7Content').get_attribute('innerHTML')

    #f_states=   pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states') 
    data1 = pd.read_html(oi_html, header=[0])[0][1::]

finally:
    driver.quit()
    

In [179]:
temp_df = pd.DataFrame()

In [180]:
comp = lambda a,b: 1 if a>= b else 0

for index, row in data1.iterrows():
    symbol = row['Symbol']
    if re.findall('NIFTY',symbol):
        continue
    
    if re.findall('KPIT',symbol):
        symbol = 'BSOFT'
        continue
        
    oi_change = row['% Chg in OI']
    stock_data = download_from_nse(symbol)
    stock_data2 = stock_data['% Dly Qt toTraded Qty']
    
    mysum = comp(stock_data2.iloc[-1],stock_data2.iloc[-2])+comp(stock_data2.iloc[-2],stock_data2.iloc[-3])+comp(stock_data2.iloc[-3],stock_data2.iloc[-4])

    temp_data = [{'symbol':stock_data.iloc[-1]['Symbol'], 'oi_%_change':oi_change, 
                 'Today -3':stock_data2.iloc[-4], 'Today -2':stock_data2.iloc[-3], 'Today -1':stock_data2.iloc[-2],'sum':mysum,
                 'Today':stock_data2.iloc[-1], 'Prev Close':stock_data.iloc[-1]['Prev Close'], 'open':stock_data.iloc[-1]['Open Price'], 'close':stock_data.iloc[-1]['Close Price'], 'Turnover':stock_data.iloc[-1]['Turnover']}]
    #print(temp_data)

    df = pd.DataFrame.from_records(temp_data, index='symbol')
    #print(df)
    temp_df = temp_df.append(df)
    #break

In [183]:
temp_df.to_csv('data/nse_filter_data.csv')

In [2]:
temp_df = pd.read_csv('data/nse_filter_data.csv')

In [3]:
raw_data = temp_df.sort_values('Turnover')

In [4]:
check1 = temp_df['close']>temp_df['open']
check2 = temp_df['close']>temp_df['Prev Close']
chk = temp_df['Today -2']>temp_df['Today -3']
chk1 = temp_df['Today -1']>temp_df['Today -2']
chk2 = temp_df['Today']>temp_df['Today -1']


temp_df['oi_%_change']

long = (temp_df['Today']>60) & (temp_df['sum']>=2)& (chk2 == True)&((check1==True) | (check2==True))
short = (temp_df['Today']<=20) & (temp_df['sum']<=1)& (chk2 == False)&((check1==False) | (check2==False))

### Buy/Sell List

In [5]:
temp_df[long]

,symbol,Prev Close,Today,Today -1,Today -2,Today -3,Turnover,close,oi_%_change,open,sum
129,MARICO,364.0,60.12,41.56,60.19,30.6,651614369.5,366.1,-1.56,362.25,2


In [6]:
temp_df[short].sort_values('Turnover', ascending=False)

,symbol,Prev Close,Today,Today -1,Today -2,Today -3,Turnover,close,oi_%_change,open,sum
195,PCJEWELLER,145.95,6.56,8.45,6.71,8.98,1.800326e+10,145.15,-11.82,149.00,1
4,INDIGO,1581.90,17.71,22.51,14.44,26.56,1.335256e+10,1554.40,16.10,1597.00,1
3,RBLBANK,680.30,14.27,24.33,26.89,27.89,2.315978e+09,675.45,17.85,684.20,0
77,ASHOKLEY,96.35,16.06,20.80,23.93,22.73,1.889255e+09,95.75,0.29,97.00,1
2,DCBBANK,202.25,9.03,16.60,28.23,25.56,1.759174e+09,202.30,19.00,203.45,1
47,PVR,1716.85,17.28,28.09,36.78,30.80,1.064478e+09,1710.05,1.63,1719.70,1
134,ULTRACEMCO,4255.05,18.22,39.11,49.00,33.08,8.972604e+08,4259.95,-1.66,4279.70,1
10,PEL,2672.95,14.83,26.73,35.38,32.72,6.302912e+08,2649.85,5.66,2681.60,1
95,BHEL,76.70,12.12,19.52,28.88,42.89,5.482854e+08,76.05,-0.27,78.00,0
74,NCC,105.85,15.72,20.62,21.90,21.13,4.782105e+08,103.90,0.34,106.00,1


In [7]:
portfolio = temp_df[short].sort_values('Turnover', ascending=False)['symbol']
portfolioToken = portfolio.apply(EQTOKEN).tolist()

In [4]:
portfolio = pd.DataFrame(downloadlist.tolist())
portfolio = pd.DataFrame(['BHARTIARTL','TCS'])

portfolioToken = portfolio[0].apply(EQTOKEN).tolist()

In [5]:
portfolio[0].iloc[-1]

'TCS'

In [6]:
papertrade = False

In [7]:
if papertrade:
    x = portfolio[0].iloc[-1]
    gc.collect()
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist.tolist())    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate)+"%")

### Visualize data

In [25]:
from talib import MIN, MAX, EMA, SMA
fig = ""
noofcandles = 60

def rsi(price, pos=1, plot=False):
    global fig
    price['RSI'] = RSI(price.close, timeperiod=20)
    
    #t1 = price[price['RSI']>=50]
    #t2 = price[price['RSI']<50]
    
    
    traceRSI = go.Scatter(x=price.index.astype('str'), y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False, fill="tonexty")
    traceLine = go.Scatter(x=price.index.astype('str'), y=np.full(price.shape[0],50),mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False)
    #traceRSI2 = go.Scatter(x=t2.index.astype('str'), y=t2['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False)
    
    if plot:
        fig.append_trace(traceLine, pos, 1)
        fig.append_trace(traceRSI, pos, 1)
        #fig.append_trace(traceRSI, pos, 1)
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="RSI"
        
    return price

def stoch(price, pos=1, plot=False):
    global fig
    price['slowk'], price['slowd'] = SMI(price)
    
    traceSK  = go.Scatter(x=price.index.astype('str'), y=price['slowk'], name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
    traceSD  = go.Scatter(x=price.index.astype('str'), y=price['slowd'], name='%D', line=dict(color='red'),showlegend=False)
    #traceLine = go.Scatter(x=price.index.astype('str'), y=np.full(price.shape[0],40),mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False, fill="tonexty")
    traceLine2 = go.Scatter(x=price.index.astype('str'), y=np.full(price.shape[0],-40),mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False, fill="tonexty")
    
    
    if plot:
        #fig.append_trace(traceLine2, pos, 1)
        fig.append_trace(traceSK, pos, 1)
        fig.append_trace(traceSD, pos, 1)
        fig['layout']['yaxis'+str(pos)]['title']="Stochastics"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
        #fig['layout']['yaxis'+str(pos)]['tickvals']=[20,80,30,70]
    return price


# Heikin-Ashi
def ha(price, pos=1, plot=False):
    global fig
    price['HAopen'] = (price.open.shift(1) + price.close.shift(1))/2
    price['HAhigh'] = price[['high','open','close']].max(axis = 1, skipna = True)
    price['HAlow'] = price[['low','open','close']].min(axis = 1, skipna = True)
    price['HAclose'] = (price.open+price.high+price.low+price.close)/4
    
    traceHA = go.Candlestick(x=price.index.astype('str'), open=price['HAopen'], high=price['HAhigh'], low=price['HAlow'], close=price['HAclose'], name="Heikin Ashi", showlegend=False)
    
    if plot:
        fig.append_trace(traceHA, pos, 1)
    return price

def annotateBuySell(price, AnnotateType=""):
    global fig
    arr = []
    for index, row in price.iterrows():
        if AnnotateType == "Buy":
            arr.append(dict(x=index,y=row['low']-1, xref='x',yref='y', ax=0, ay=35,
                                       showarrow=True, arrowhead=1,arrowsize=0.5, arrowwidth=7,arrowcolor='green'))
        elif AnnotateType == "Sell":
            arr.append(dict(x=index,y=row['high']+1, xref='x',yref='y', ax=0, ay=-35,
                                       showarrow=True, arrowhead=1,arrowsize=0.5, arrowwidth=7,arrowcolor='red'))
        else:
            arr.append(dict(x=index,y=row['low']-1, xref='x',yref='y', ax=0, ay=50, text=row["text"],
                                       showarrow=True, arrowhead=1,arrowsize=1, arrowwidth=1,arrowcolor='black'))
            
    
    arr.extend(fig['layout']['annotations'])
    fig['layout']['annotations']=arr
    return price

def SMI(price):
        k_period = 40
        d_period = 10
        kma = 3
        dma = 3
    
        LOW = price['low']
        HIGH = price['high']
        CLOSE = price['close']
        
        ll = MIN(LOW, timeperiod=k_period)
        hh = MAX(HIGH, timeperiod=k_period)
        diff = hh - ll
        rdiff = CLOSE - (hh+ll)/2
        # Nested Moving Average for smoother curves
        avgrel = EMA(EMA(rdiff,d_period),d_period)
        avgdiff = EMA(EMA(diff,d_period),d_period)
        # SMI calculations
        #SMI = avgdiff != 0 ? (avgrel/(avgdiff/2)*100) : 0
        
        #SMI = 0
        #if  avgdiff != 0:
        
        SMI = (avgrel/(avgdiff/2)*100)
        
        SMIsignal = EMA(SMI,d_period)
        
        return SMI, SMIsignal

def plot_set2(temp_data, symbol, buy,sell):
    global fig
    fig = tools.make_subplots(rows=3, cols=1, shared_xaxes=True, row_width=[1,1,2], vertical_spacing = 0.01)
    fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="bottom") #, range=[xMin,xMax])
    fig['layout'].update(height=950, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
    #fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['anchor'] = 'x'
    fig['layout']['yaxis']['side'] = 'right'
    
    fig['layout']['xaxis']['rangeselector'] = dict(
                buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                              dict(count=3, label='3h', step='hour', stepmode='backward'),
                              dict(count=7, label='1d', step='hour', stepmode='backward'),
                              dict(step='all')]))
    
    noofcandles = 74
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['xaxis']['range'] = [xMin, xMax]

    temp_data = ha(temp_data,1,True)
    temp_data = rsi(temp_data,3, True)
    temp_data = stoch(temp_data,2, True)
    
    init_notebook_mode(connected=True)
    
    annotateBuySell(buy, "Buy")
    annotateBuySell(sell, "Sell")
    
    plot(fig, filename="plot/"+symbol+"test.html")



In [57]:
symbol = 'BHEL'
temp_data = getData(symbol,dt.datetime(2019,4,18,9,0),dt.datetime.now(),freq="5minute")

In [35]:
temp_data.tail()

,close,high,low,open,volume
date,,,,,
2019-04-22 11:47:00,134.60,134.75,133.95,133.95,150036
2019-04-22 11:48:00,134.50,134.85,134.40,134.60,55132
2019-04-22 11:49:00,134.35,134.60,134.15,134.60,20248
2019-04-22 11:50:00,134.30,134.40,134.20,134.40,14909
2019-04-22 11:51:00,134.35,134.35,134.20,134.20,3565


### Plots

In [ ]:
for symbol in portfolio:
    print(symbol)
    temp_data = getData(symbol,dt.datetime(2019,4,18,9,0),dt.datetime.now(),freq="5minute")
    a = algoTrade(temp_data)
    #buy, sell = a.tradeDecision()
    plot_set2(temp_data, symbol, pd.DataFrame(), pd.DataFrame())
    

### Trading Strategy

In [ ]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
UP = 0
DOWN = 1
iRSI = 0
STATE = "SELL"
Fast_K = 0
Slow_K = 0
Slow_D = 0

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a, days: ROC(a,days) >= 0.01
TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
#TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)))
#TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2)))

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0


from talib import MIN, MAX
class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, iRSI, sma20, sma50, Fast_K, Slow_K, Slow_D
        
        OPEN = (price.open.shift(1) + price.close.shift(1))/2
        HIGH = price[['high','open','close']].max(axis = 1, skipna = True)
        LOW = price[['low','open','close']].min(axis = 1, skipna = True)
        CLOSE = (price.open+price.high+price.low+price.close)/4
        
        Slow_K, Slow_D = SMI(price)
        iRSI = RSI(CLOSE, timeperiod=20)
    
    def long_pa(self):
        
        temp_df = (REF(CLOSE,1) - REF(LOW,1))/REF(CLOSE,1) <= 0.0005
        
        #print( (REF(CLOSE,1) - REF(LOW,1))/REF(CLOSE,1))
        #temp_df = temp_df & CROSSOVER(Slow_K, Slow_D)
        
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_pa(self):
        temp_df = (REF(HIGH,1) - REF(OPEN,1))/REF(OPEN,1) <= 0.00000001
        #temp_df = temp_df & CROSSOVER(Slow_D, Slow_K)
        
        return pd.DataFrame( temp_df , columns=["sell"])
 
    
    def tradeDecision(self):
        buy  = self.long_pa() # |self.long_macd_crossover() # | self.long_bb() #| self.long_ichimoku() #| self.long_bull_engulf_ema() #| self.long_ichimoku()
        
        sell = self.short_pa() # |self.short_macd_crossover() # | self.short_bb() #| self.short_bear_engulf() #| self.short_hanging_man() #|  self.short_ichimoku()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE
        
        buy['sl'] = MIN(REF(LOW,1),timeperiod=2)
        sell['sl'] = MAX(REF(HIGH,1),timeperiod=2)
        buy['so'] = CLOSE * 1.01
        sell['so'] = CLOSE * 1.01

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
        
    def tradeBot(self, symbol):
        global STATE
        buy, sell = self.tradeDecision()
        logger.debug(symbol+","+STATE)
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        
        price = CLOSE[-1]
        trigger = CLOSE[-1]
        #stoploss = SD[-1]
        stoploss = 0
        squareoff = 3 * SD[-1] 
        
        logger.debug(buy)
        logger.debug(sell)
        
        if buy.empty | sell.empty:
            return
        
        if (buy.empty != True) & ((delT1 == buy.index[-1]) | (delT2 == buy.index[-1])) & (STATE != "SELL"):
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_sl(symbol, price, trigger, quantity=1)
            STATE = "SELL"
        elif (sell.empty != True) & ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) & (STATE != "BUY"):
            STATE = "BUY"
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_sl(symbol, price, trigger, quantity=1)
            
    
            
        
papertrade = False
if True:
    global fig
    noofcandles = 60
    x = "BHEL"
    #toDate = dt.datetime(2019,4,18,3,30)
    toDate = dt.datetime.now()
    fromDate = dt.datetime(2019,4,18,9,0)
    temp_data = downloadData(x, fromDate = fromDate, toDate=toDate, freq="5minute")
    
    
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    #annotateBuySell(buy, "Buy")
    #annotateBuySell(sell, "Sell")
    
    plot_set2(temp_data, x, pd.DataFrame(),pd.DataFrame())

In [2]:
papertrade = True

In [8]:
if mode == "Live":
    kws.connect(threaded=True)

In [21]:
if mode == "Live":
    print(kws.is_connected())

False


In [20]:
if mode == "Live":
    kws.close()

In [ ]:
if mode == "Live":
    print(kws.is_connected())

In [23]:
kws.tradebook_df.loc['BHARTIARTL','symbol'].CLOSE

AttributeError: 'algoTrade' object has no attribute 'CLOSE'